The Black-Scholes is a mathematical method to model the dynamics of a derivative instrument. Its formula estimates the theoretical value of European-style options and shows that the option has a unique price given the security and its expected return.

In the below cell, we are querying price data for 1 year for a sample stock in our case Tesla from Yahoo Finance.

In [40]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader

dt = datetime.datetime.now()
dt_m1 = dt - datetime.timedelta(365)

df = DataReader('TSLA', 'yahoo', dt_m1, dt).sort_values('Date')
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-09-30,263.043335,258.333344,260.333344,258.493347,53868000.0,258.493347
2021-10-01,260.260010,254.529999,259.466675,258.406677,51094200.0,258.406677
2021-10-04,268.989990,258.706665,265.500000,260.510010,91449900.0,260.510010
2021-10-05,265.769989,258.066681,261.600006,260.196655,55297800.0,260.196655
2021-10-06,262.220001,257.739990,258.733337,260.916656,43898400.0,260.916656
...,...,...,...,...,...,...
2022-09-23,284.500000,272.820007,283.089996,275.329987,63615400.0,275.329987
2022-09-26,284.089996,270.309998,271.829987,276.010010,58076900.0,276.010010
2022-09-27,288.670013,277.510010,283.839996,282.940002,61925200.0,282.940002


Loosely defined, the Black-Scholes call option price is calculated by subtracting the present value of the strike price $K$ multiplied by the risk-adjusted probability that the option will be exercised $N(d_2)$, of the product of the stock price $S$ by the probability of receiving the stock at the expiration of the option contract $N(d_1)$.

$$C = SN(d_1) - K e^{-rt} N(d_2)$$

$$d_1 = \frac{ln_K^S + (r+ \frac{\sigma_v^2}{2} t) }{\sigma_s * \sqrt{t}}$$

$$d_2 = d_1 - \sigma_s \sqrt{t}$$

where:
1. $C$ - Call option price
2. $S$ - Current stock price (or other underlying instrument)
3. $K$ - Strike price
4. $r$ - Risk-free interest rate
5. $t$ - Time to maturity
6. $N$ - Normal distribution
7. $N(d_1)$ - Probability of receiving the stock at the expiration of our option contract. (The expected value of the receipt of the stock is contingent on the exercise of the option. It is, therefore, the product of the conditional expected value of the receipt of the stock given that exercise has occurred times the probability of exercise)
8. $N(d_2)$ - Risk-adjusted probability that the option will be exercised

In [39]:
df['rel_day_returns'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)
sigma = np.sqrt(df.index.unique().shape[0]) * df['rel_day_returns'].std()